In [1]:
import cv2, json, time
import numpy as np

from custom_utils import *

In [2]:
data_path = './CAMPUS/Parkinglot/view-GL2', './CAMPUS/Parkinglot/view-GL5'

In [5]:
json_dts = [read_json('{}.json'.format(i)) for i in data_path]
vids     = [cv2.VideoCapture('{}.mp4'.format(i)) for i in data_path]

frame_idx = 0

obj_dts1 = json_dts[0]
obj_dts2 = json_dts[1]

while(True):
    # 이미지 read
    ret1, image_cam1 = vids[0].read() # (h, w, 3)
    ret2, image_cam2 = vids[1].read() 
    
    image_cam1 = cv2.resize(image_cam1, (640, 360)) # (w, h)
    image_cam2 = cv2.resize(image_cam2, (640, 360))

    objs1 = obj_dts1[str(frame_idx)]
    objs2 = obj_dts2[str(frame_idx)]
    
    ## Trackers
    trks1 = convert_trks_shape(objs1)
    trks2 = convert_trks_shape(objs2)
    
    draw_rectangle(image_cam1, trks1)
    draw_rectangle(image_cam2, trks2)

    ## find similarity matrix
    similarity_matrix = calculate_similarity_matrix(trks1, trks2)
    
    ## find match, unmatch use Hungarian algorithm
    assignments, cos_similarities = calculate_hungarian_algorithm(similarity_matrix)
    matched   = assignments[cos_similarities > 0.5]
    unmatched = calculate_non_matched(trks1, trks2, matched)
    
    ## print matched/unmatched info 
    print('----[ {:5d} frame matched/unmatched info ----]'.format(frame_idx))
    
    print('matched')
    print(matched)
    
    print('unmatched_cam1')
    print(unmatched[0])
    
    print('unmatched_cam2')
    print(unmatched[1], end='\n'*2)
    
    
    if not ret1 or not ret2: 
        break
    
    # Display
    full_cam = np.concatenate([image_cam1, image_cam2], axis=0)
    cv2.imshow('full_cam', full_cam)
    
    
    
    frame_idx += 1
    # time.sleep(0.1)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break


vids[0].release()
vids[1].release()
cv2.destroyAllWindows()